# Hands-on with Spark - Google Dataproc

![spark](https://cdn-images-1.medium.com/max/300/1*c8CtvqKJDVUnMoPGujF5fA.png)

PySpark is the Python API of Spark; which means it can do almost all the things python can- Machine learning (ML), exploratory data analysis (EDA), ETLs for data platform. And all of them in a distributed manner.

![pyspark](https://editor.analyticsvidhya.com/uploads/20981sp3.JPG)

In simple terms, each time you submit a PySpark job, the code gets internally converted into a MapReduce program and gets executed in the Java Virtual Machine. Spark also uses Lazy Evaluation, it delays its evaluation as much as it can. Each time you submit a job, spark creates an action plan for how to execute the code, and then does nothing. Finally, when you ask for the result (i.e, calls an action), it executes the plan, which is basically all the transofrmations you have mentioned in your code.


For our lesson, we will be running Spark on **local** mode.

In this mode, Spark runs on a single machine, utilizing available cores (similar to Polars).
It's useful for development, learning, testing, and debugging since everything runs on the local machine without needing a cluster.

In actual production environments, Spark is usually run on a cluster of machines (YARN, Mesos or Kubernetes mode).

## Initializing Spark

# For mult-node spark, call Spark session with Yarn 

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DistributedSparkApp") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

Invoking `spark` will print the SparkContext

In [ ]:
spark

## Spark SQL and DataFrames

Spark SQL and DataFrames are higher-level modules of Apache Spark, they work together to make data processing with Spark more intuitive and optimized, especially for those who come from an SQL or data analytics background.

- **SQL Interface to Spark**: Spark SQL lets users execute SQL queries alongside Spark programs.

- **DataFrames are abstraction over RDDs**: A DataFrame is a distributed collection of data organized into named columns. Conceptually, it's equivalent to a table in a relational database or a data frame in R or Python, but with optimizations for distributed processing and scalability. While RDD (Resilient Distributed Dataset) is a fundamental data structure in Spark, DataFrames provide a higher-level abstraction that is often easier to use and more optimized for many tasks.

### DataFrame Creation

In [ ]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

In [ ]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

PySpark infers the DataFrame schema (dtypes) by taking a sample from the data (just like Pandas)

In [ ]:
df.show(5)


You can also pass an explicit schema:

In [ ]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

PySpark DataFrame is lazily evaluated and invoking `df` does not trigger the computation and show anything. You need to explicitly call the `show` method:

In [ ]:
df.show()
df.printSchema()

### Getting Data In/Out

There are many data sources available in PySpark such as CSV, Parquet, ORC, JDBC, text, Avro, etc.

First, let's download the files in various format from GCS, data which we explored in Unit 2.3 and 2.8.

In [ ]:
!mkdir -p data

In [ ]:
!wget -P data https://storage.googleapis.com/su-artifacts/movies_metadata.csv
!wget -P data https://storage.googleapis.com/su-artifacts/ratings.csv

In [ ]:
!wget -P data https://storage.googleapis.com/su-artifacts/taxi_trip_data.csv

In [ ]:
!wget -P data https://storage.googleapis.com/su-artifacts/userdata1.orc
!wget -P data https://storage.googleapis.com/su-artifacts/userdata1.parquet

In [ ]:
!ls /data

## Move downloaded files from local disk to hadoop file system

In [ ]:
!hdfs dfs -put /data /user/

In [ ]:
!hdfs dfs -ls /user/data

In [ ]:
$

In [ ]:
df = spark.read.parquet('/user/data/userdata1.parquet')
df.show()

In [ ]:
df = spark.read.orc('/user/data/userdata1.orc')
df.show()

In [ ]:
movies = spark.read.csv('/user/data/movies_metadata.csv', header=True)
ratings = spark.read.csv('/user/data/ratings.csv', header=True)
taxi = spark.read.csv('/user/data/taxi_trip_data.csv', header=True)

In [ ]:
movies.printSchema()

In [ ]:
ratings.printSchema()

In [ ]:
taxi.printSchema()

In [ ]:
movies = spark.read.csv('/user/data/movies_metadata.csv', header=True, inferSchema=True, quote="\"", escape="\"")

In [ ]:
movies.printSchema()

In [ ]:
ratings = spark.read.csv('/user/data/ratings.csv', header=True, inferSchema=True)

In [ ]:
ratings.printSchema()

Passing a schema will speed up the read.

In [ ]:
dtypes = ['vendor_id integer',
 'pickup_datetime timestamp',
 'dropoff_datetime timestamp',
 'passenger_count integer',
 'trip_distance double',
 'rate_code integer',
 'store_and_fwd_flag string',
 'payment_type integer',
 'fare_amount double',
 'extra double',
 'mta_tax double',
 'tip_amount double',
 'tolls_amount double',
 'imp_surcharge double',
 'total_amount double',
 'pickup_location_id integer',
 'dropoff_location_id integer']

In [ ]:
taxi = spark.read.csv('/user/data/taxi_trip_data.csv', header=True,
                      schema=', '.join(dtypes))

In [ ]:
taxi.printSchema()

There are a couple of ways to view your dataframe in PySpark:

1.   `df.take(5)` will return a list of five Row objects.
2.   `df.collect()` will get all of the data from the entire DataFrame. Be really careful when using it, because if you have a large data set, you can easily crash the driver node.
3.   `df.show()` is the most commonly used method to view a dataframe. There are a few parameters we can pass to this method, like the number of rows and truncaiton. For example, `df.show(5, False)` or ` df.show(5, truncate=False)` will show the entire data wihtout any truncation.
4.   `df.limit(5)` will **return a new DataFrame** by taking the first n rows. As spark is distributed in nature, there is no guarantee that `df.limit()` will give you the same results each time.

In [ ]:
movies.show(10)

In [ ]:
movies.show(5, truncate=False)

In [ ]:
# can also be shown vertically
movies.show(1, vertical=True, truncate=False)

In [ ]:
ratings.show(10)

In [ ]:
taxi.show(10)

In [ ]:
taxi.limit(5).show()

In [ ]:
movies.describe().show()

In [ ]:
ratings.describe().show()

In [ ]:
taxi.describe().show()

### DataFrame Operations on Columns

We will go over the following in this section:

1.   Selecting a Column
2.   Selecting Multiple Columns
3.   Adding New Columns
4.   Renaming Columns
5.   Removing Columns


In [ ]:
movies.id

Just like DataFrame, selecting a column is lazily evaluated and does not trigger the computation but returns a `Column` instance.

It can be used to select columns and returns another DataFrame:

In [ ]:
movies.select(movies.id).show()

In [ ]:
movies.select(movies.title, movies.overview).show(truncate=False)

Creating a new column:

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
# lit means literal. It populates the row with the literal value given
ratings.withColumn('review', lit('Great movie!')).show(10)

In [ ]:
ratings.withColumn('review', lit('Great movie!')) \
       .withColumn('mood', lit(5)) \
       .show(10)

In [ ]:
movies.withColumnRenamed('overview', 'summary').show(5)

In [ ]:
ratings.drop('timestamp').show()

### Common Transformation Functions

In [ ]:
# Functions available in PySpark
from pyspark.sql import functions
# We can use the dir function to view the available functions
print(dir(functions))

String functions:

In [ ]:
from pyspark.sql.functions import concat, lower, upper, substring

In [ ]:
movies.show(5)

In [ ]:
movies.select(movies.title, movies.tagline, upper(movies.title), lower(movies.tagline),
              substring(movies.overview, 1, 10),
              concat(movies.title, lit(' Part 1')).alias('new_title')
              ).show()

Numeric functions:

In [ ]:
from pyspark.sql.functions import mean, max, round

In [ ]:
ratings.select(mean(ratings.rating), max(ratings.rating)).show()

In [ ]:
ratings.withColumn('new_rating', round(ratings.rating)).show()

Date/time functions:

In [ ]:
from pyspark.sql.functions import to_date, to_timestamp, datediff

In [ ]:
ratings.withColumn('new_timestamp', to_timestamp(ratings.timestamp)).show()

Replace the unix timestamp with new timestamp:

In [ ]:
ratings = ratings.withColumn('timestamp', to_timestamp(ratings.timestamp))

In [ ]:
ratings.show()

Computing the difference in minutes is more tedious, we'll need to convert the timestamps to unix timestamps (seconds since epoch), compute the difference, and divide by 60.

In [ ]:
taxi = taxi.withColumn('trip_duration', (taxi.dropoff_datetime.cast("long") - taxi.pickup_datetime.cast("long"))/60)

In [ ]:
taxi.show(10)

### User-Defined Functions (UDF)

PySpark User-Defined Functions (UDFs) help you convert your Python code into a scalable version of itself. It is handy, but beware, as the performance is slower compared to PySpark functions.

In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

In [ ]:
@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return (series + 1.0).astype(float)

In [ ]:
taxi.select(taxi.passenger_count, pandas_plus_one(taxi.passenger_count)).show()

### DataFrame Operations on Rows

We will show the follwoing in this section:

1.   Filtering Rows
2. 	 Get Distinct Rows
3.   Sorting Rows

In [ ]:
movies.filter(movies.status == 'In Production').show()

In [ ]:
ratings.filter(ratings.rating > 4.5).show()

Use `&` and `|` for combining conditions as `and` and `or`:

In [ ]:
ratings.filter((ratings.rating < 1.5) | (ratings.rating > 4.5)).show()

> 1. Filter `movies` for `status` to be `In Production` and `vote_average` greater than 6.
> 2. Filter `taxi` for `trip_duration` greater than 1 hour, `trip_distance` greater than 10 miles and `passenger_count` equal to and less than 2.

In [ ]:
# 1. Filter movies for status 'In Production' and vote_average > 6
# This will show movies that are still in production and have a vote_average greater than 6.
# If this returns no records, it means there are no such movies in your dataset.
movies.filter(
    (movies.status == 'In Production') & (movies.vote_average > 6)
).show()

In [ ]:
# 2. Filter taxi for trip_duration > 1 hour, trip_distance > 10 miles, and passenger_count <= 2
# First, create the trip_duration column (in minutes) using pickup and dropoff timestamps.
from pyspark.sql.functions import col

taxi = taxi.withColumn(
    'trip_duration',
    (col('dropoff_datetime').cast('long') - col('pickup_datetime').cast('long')) / 60
)

# Now filter for trips longer than 1 hour, distance > 10 miles, and 2 or fewer passengers.
taxi.filter(
    (col('trip_duration') > 60) &
    (col('trip_distance') > 10) &
    (col('passenger_count') <= 2)
).show()

In [ ]:
movies.select('status').distinct().show()

In [ ]:
taxi.orderBy('trip_duration').show()

In [ ]:
taxi.orderBy('trip_distance', ascending=False).show()

### Grouping Data

In [ ]:
movies.groupBy('status').count().show()

In [ ]:
ratings.groupBy('movieId').avg('rating').show()

### Joining DataFrames

In [ ]:
ratings_avg_by_movie = ratings.groupBy('movieId').avg('rating')

In [ ]:
ratings_avg_by_movie.columns

In [ ]:
movies.join(ratings_avg_by_movie, movies.id == ratings.movieId, 'inner').select('title', 'vote_average', 'avg(rating)').show()

### Spark SQL

DataFrame and Spark SQL share the same execution engine so they can be interchangeably used seamlessly. For example, you can register the DataFrame as a table and run a SQL easily:

In [ ]:
taxi.createOrReplaceTempView("taxi")
spark.sql("SELECT count(*) from taxi").show()

In addition, UDFs can be registered and invoked in SQL out of the box:

In [ ]:
@pandas_udf("double")
def div(s1: pd.Series, s2: pd.Series) -> pd.Series:
    return s1 / s2

In [ ]:
spark.udf.register("div", div)
spark.sql("SELECT trip_distance, trip_duration, div(trip_distance, trip_duration) AS speed FROM taxi").show()

In [ ]:
taxi.selectExpr("div(trip_distance, trip_duration)").show()